<a href="https://colab.research.google.com/github/ryar001/tester/blob/master/portfolio_optimise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting and setting directory


In [14]:
# mounting the google drive 
# set the dirrectory to the location i wanted
# relative path so shld work
# since the cwd is now set to the this folder, we can import item from there
from google.colab import drive
drive.mount('/content/drive')
%cd './drive/My Drive/Colab Notebooks'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks


In [35]:
# add a path to '/content/drive/My Drive/My python files/lib'
# as i store the self use py files

import sys
sys.path.append(str(Path(Path.cwd().parent,'My python files/lib')))

In [32]:
from pathlib import Path
Path(Path.cwd().parent,'My python files/lib')

PosixPath('/content/drive/My Drive/My python files/lib')

In [33]:
import os
os.listdir(Path(Path.cwd().parent,'My python files/lib'))

['johansen.py',
 'Pickle.py',
 'stock_get.py',
 'A_st.py',
 'Forex_pair.py',
 '__pycache__']

In [15]:
import pathlib
from pathlib import
pathlib.Path.cwd()

PosixPath('/content/drive/My Drive/Colab Notebooks')

In [37]:
sys.path

['./content/drive/My Drive/My python files/lib',
 '',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 './content/drive/My Drive/My python files/lib',
 '/content/drive/My Drive/My python files/lib']

In [38]:
import stock_get

# New Section

In [ ]:
!pip install git+https://github.com/ryar001/BT-package-test


  Cloning https://github.com/ryar001/BT-package-test to c:\users\runmi\appdata\local\temp\pip-req-build-dfcouzvo


  Running command git clone -q https://github.com/ryar001/BT-package-test 'C:\Users\runmi\AppData\Local\Temp\pip-req-build-dfcouzvo'
  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git clone -q https://github.com/ryar001/BT-package-test 'C:\Users\runmi\AppData\Local\Temp\pip-req-build-dfcouzvo'
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, SPY


class SmaCross(Strategy):
    def init(self):
        Close = self.data.Close
        self.ma1 = self.I(SMA, Close, 10)
        self.ma2 = self.I(SMA, Close, 20)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(GOOG, SmaCross,
              cash=10000, commission=.002)
bt.run()
bt.plot()

ImportError: ignored

In [ ]:
tyututy

NameError: ignored